# Lab 4: Parte 2 - Optimización multiobjetivo en problemas no lineales

- Jaime Andres Torres Bermejo - 202014866
- Elkin Rafael Cuello - 202215037

In [1]:
import matplotlib.pyplot as plt
import math
from pyomo.environ import *
from pyomo.opt import SolverFactory
solver = SolverFactory('ipopt')
import pandas as pd
import numpy as np

## Pregunta 1: Beneficio vs Unidades Vendidas

Una startup necesita aumentar las ventas de un producto a través de publicidad. Si la empresa gasta $a$ (medido en miles de dólares) en publicidad y cobra un precio de $p=10+0.38a$ dólares por unidad, entonces puede vender $1000-10p+20\sqrt{a}$ unidades del producto. El costo de producción por unidad del producto es $6$. Ayuda a la empresa a encontrar al menos 10 soluciones de Pareto óptimas y traza la frontera de Pareto entre los objetivos de maximizar el beneficio y maximizar el número de unidades vendidas.

### Parte I: Formulación del modelo matemático

Formula el problema matemático listando:
- Variables de decisión
- Función objetivo
- Restricciones

1. Variables de Decisión  
  - $ a = $ inversión en publicidad (miles de dólares)

2. Función Objetivo  
  - $Max :\ ventas\ unitarias(a) = f_1(a) $  
    - $ventas\ unitarias(a) = 1000-10p+20\sqrt{a}$  
    - Sustituyendo:  
      - $ ventas\ unitarias(a) = 1000-10(10+0.38a)+20\sqrt{a} $  
      - $ ventas\ unitarias(a) = 890-3.8a+20\sqrt{a} $  
    - $ Max :\ ventas\ unitarias(a) = f_1(a) = 890-3.8a+20\sqrt{a} $
    
  - $Max :\ beneficio(a) = f_2(a) $  
    - $ beneficio = ventas\ unitarias * margen - inversión$
    - $ beneficio = ventas\ unitarias * (precio - 6) - a $
    - Sustituyendo:  
      - $ beneficio(a) = ventas\ unitarias * ((10+0.38a) - 6) - a $
      - $ beneficio(a) = (890-3.8a+20\sqrt{a}) * (4+0.38a) - a $
    - $Max :\ beneficio(a) = f_2(a) = -1.444a^2 + 7.6\sqrt{a}*a + 80\sqrt{a} + 322a +3560 $

3. Restricciones  
  - $ a > 0 $

In [2]:

solver = SolverFactory('ipopt')

def construir_modelo():
    modelo = ConcreteModel()
    
    modelo.inversion_publicitaria = Var(bounds=(1e-6, None), initialize=1, within=NonNegativeReals)
    
    modelo.ventas = Expression(expr=890 - 3.8 * modelo.inversion_publicitaria + 20 * sqrt(modelo.inversion_publicitaria))
    
    modelo.rentabilidad = Expression(
        expr=-1.444 * (modelo.inversion_publicitaria ** 2) + 
        7.6 * modelo.inversion_publicitaria * sqrt(modelo.inversion_publicitaria) +
        80 * sqrt(modelo.inversion_publicitaria) + 322 * modelo.inversion_publicitaria + 3560
    )
    
    return modelo

### Parte II: Hayar los extremos de Pareto

Encontrar tanto la venta máxima como la ganancia máxima de unidades vendidas.

> Puedes usar `model.res1.deactivate()` para desactivar la restricción para esto


In [ ]:
def optimizar_rentabilidad():
    modelo = construir_modelo()
    modelo.objetivo_rentabilidad = Objective(expr=modelo.rentabilidad, sense=maximize)
    solucion = solver.solve(modelo)
    
    if (solucion.solver.status == SolverStatus.ok) and (solucion.solver.termination_condition == TerminationCondition.optimal):
        rentabilidad_max = modelo.rentabilidad()
        publicidad_optima = modelo.inversion_publicitaria()
        ventas_optimas = modelo.ventas()
        return rentabilidad_max, publicidad_optima, ventas_optimas
    else:
        raise ValueError('No se pudo obtener una solución óptima para maximizar la rentabilidad.')

def optimizar_ventas():
    modelo = construir_modelo()
    modelo.objetivo_ventas = Objective(expr=modelo.ventas, sense=maximize)
    solucion = solver.solve(modelo)
    
    if (solucion.solver.status == SolverStatus.ok) and (solucion.solver.termination_condition == TerminationCondition.optimal):
        ventas_max = modelo.ventas()
        publicidad_optima = modelo.inversion_publicitaria()
        rentabilidad_optima = modelo.rentabilidad()
        return ventas_max, publicidad_optima, rentabilidad_optima
    else:
        raise ValueError('No se encontró una solución óptima para maximizar las ventas.')

try:
    rentabilidad_max, publicidad_rentabilidad, ventas_rentabilidad = optimizar_rentabilidad()
    print(f'Rentabilidad máxima: {rentabilidad_max}, Ventas: {ventas_rentabilidad}, Inversión publicitaria: {publicidad_rentabilidad}')
    
    ventas_max, publicidad_ventas, rentabilidad_ventas = optimizar_ventas()
    print(f'Ventas máximas: {ventas_max}, Rentabilidad: {rentabilidad_ventas}, Inversión publicitaria: {publicidad_ventas}')

except ValueError as e:
    print(e)

### Parte III: Graficar la frontera de Pareto

Encuentra al menos 10 puntos de Pareto-óptimos y traza la frontera de Pareto entre los objetivos de maximizar el beneficio y maximizar el número de unidades vendidas.

Implementen el método de **$\epsilon$-constraint** para encontrar el **frente óptimo de Pareto** en un problema de optimización multiobjetivo. La idea detrás de este método es optimizar un objetivo principal mientras se aplica una restricción sobre el otro objetivo, permitiendo obtener soluciones óptimas que representen diferentes compromisos entre ambos objetivos (en este caso, maximizar el **beneficio** y maximizar las **ventas unitarias**).

### Explicación del Método $\epsilon$-Constraint

1. **Definición del Problema**:  
   El problema tiene dos objetivos conflictivos:
   - **Maximizar el beneficio** ($f_2(a)$).
   - **Maximizar las ventas unitarias** ($f_1(a)$).
   
   Como estos objetivos están en conflicto (aumentar el beneficio podría reducir las ventas y viceversa), necesitamos encontrar un conjunto de soluciones óptimas donde no se puede mejorar un objetivo sin empeorar el otro, es decir, el **frente de Pareto**.

2. **¿Qué es el método $\epsilon$-Constraint?**:  
   El método $\epsilon$-constraint optimiza uno de los objetivos (por ejemplo, el beneficio) mientras que el otro objetivo se convierte en una restricción limitada por un valor $\epsilon$.
   
   En el código, lo que se hace es:
   - Primero, se **maximiza el beneficio** bajo una restricción que lo limita a un porcentaje de su valor máximo posible.
   - Luego, se optimiza la función de **ventas unitarias** asegurando que el beneficio no disminuya por debajo del valor encontrado en la primera etapa.

3. **Cómo se calculan los valores $\epsilon$ (fronteras)**:  
   Los valores $\epsilon$ en este caso se calculan como un porcentaje de la diferencia entre el **valor máximo** y el **valor mínimo** del beneficio. Este valor $\epsilon$ establece el límite superior para el beneficio en cada iteración. Así es como se asegura que el beneficio no supere un porcentaje del beneficio máximo posible, permitiendo así maximizar las ventas unitarias bajo ese límite.

   La fórmula para establecer este límite es:
   $$
   \text{profit\_limit} = \text{profit\_max} - (\text{profit\_max} - \text{profit\_min}) \times w1
   $$
   Donde:
   - **profit\_max** es el valor máximo del beneficio.
   - **profit\_min** es el valor mínimo del beneficio.
   - **w1** es un peso que varía entre 0 y 1
4. **Iteración para encontrar soluciones de Pareto**:  
   El código genera múltiples soluciones cambiando el valor de **w1** en el rango de 0 a 1. Esto permite explorar diferentes compromisos entre el beneficio y las ventas unitarias:
   - Cuando **w1 = 0**, se permite el beneficio máximo posible.
   - Cuando **w1 = 1**, se reduce el beneficio al mínimo y se maximizan las ventas.
   
   Cada vez que el valor de **w1** cambia, el modelo se resuelve nuevamente para encontrar un nuevo compromiso óptimo entre beneficio y ventas, generando así una solución del frente de Pareto.

In [ ]:
def frontera_pareto(rentabilidad_max, rentabilidad_min, pesos_w1):
    lista_rentabilidad = []
    lista_ventas = []
    
    for w1 in pesos_w1:
        umbral_rentabilidad = rentabilidad_max - (rentabilidad_max - rentabilidad_min) * w1
        modelo = construir_modelo()
        
        modelo.objetivo_ventas = Objective(expr=modelo.ventas, sense=maximize)
        
        modelo.restriccion_rentabilidad = Constraint(expr=modelo.rentabilidad >= umbral_rentabilidad)
        
        solucion = solver.solve(modelo)
        
        if (solucion.solver.status == SolverStatus.ok) and (solucion.solver.termination_condition == TerminationCondition.optimal):
            lista_rentabilidad.append(modelo.rentabilidad())
            lista_ventas.append(modelo.ventas())
        else:
            print(f'No se encontró una solución óptima para w1 = {w1}')
            lista_rentabilidad.append(None)
            lista_ventas.append(None)
    
    lista_rentabilidad = [r for r in lista_rentabilidad if r is not None]
    lista_ventas = [v for v in lista_ventas if v is not None]
    
    return lista_rentabilidad, lista_ventas

def dibujar_frontera_pareto(lista_rentabilidad, lista_ventas):
    plt.figure(figsize=(10,6))
    plt.plot(lista_rentabilidad, lista_ventas, 'o-', color='darkorange', linewidth=2, markerfacecolor='blue', markeredgewidth=2, markeredgecolor='black', label='Soluciones')
    plt.xlabel('Rentabilidad', fontsize=12, fontweight='bold')
    plt.ylabel('Ventas', fontsize=12, fontweight='bold')
    plt.title('Frontera de Pareto: Rentabilidad vs Ventas', fontsize=14, fontweight='bold')
    plt.grid(True, linestyle='--', linewidth=0.7, color='gray', alpha=0.7)
    plt.legend(loc='best', fontsize=12)
    plt.show()

pesos_w1 = np.linspace(0, 1, 10)
lista_rentabilidad, lista_ventas = frontera_pareto(rentabilidad_max, rentabilidad_ventas, pesos_w1)

dibujar_frontera_pareto(lista_rentabilidad, lista_ventas)

### Parte (d): Discusión

Supongamos que la empresa startup ahora decide que, aunque ambos objetivos son importantes, maximizar el número de unidades vendidas es más importante que maximizar el beneficio. ¿Qué sugerencias le darías a la empresa basándote en tus resultados de las partes (b) y (c)?

#### **Respuesta:**

Basándome en los resultados obtenidos en las partes (b) y (c), si la empresa valora más maximizar el número de unidades vendidas, le sugeriría enfocarse en soluciones donde el objetivo de ventas se priorice en la frontera de Pareto. Específicamente, podría elegir estrategias de inversión publicitaria que se encuentren en el extremo de la curva de Pareto donde las ventas son mayores, aceptando un posible sacrificio en la rentabilidad. Sin embargo, es importante que la empresa evalúe cuidadosamente hasta qué punto puede reducir la rentabilidad sin comprometer su sostenibilidad a largo plazo, de forma que encuentre un equilibrio.